# Empty Weight

This section describes the code used for computing the empty weight fraction for a given MTOW and fuel weight, using various aircraft parameters. This module uses refined weight estimation relations described in [weight and balance section](../weight_and_balance/weight.ipynb). Below code defines a set of functions for computing weight of individual components:

In [1]:
import numpy as np
import nbimporter
from parameters import AircraftParameters
from fuel import compute_fuel_weight

def compute_wing_weight(parameters:AircraftParameters, MTOW):

    A = parameters.A
    Wfw = parameters.fuel_weight
    Sw = parameters.S
    sweep = np.radians(parameters.sweep_qc)
    taper_ratio = parameters.taper_ratio_wing
    Nz = parameters.Nz
    tc = parameters.tc_avg
    q = 0.5 * parameters.rho_cruise * parameters.cruise_speed**2

    wing_weight = 0.036*Sw**0.758 * Wfw**0.0035 * (A/np.cos(sweep)**2)**0.6 * q**0.006 * taper_ratio**0.04 \
    * (100*tc/np.cos(sweep))**-0.3 * (Nz*MTOW)**0.49

    return wing_weight

def compute_fuselage_weight(parameters: AircraftParameters, Wdg: float):

    Sf = parameters.Sf
    D = parameters.D
    Lf = parameters.Lf
    Lt = parameters.Lt
    Nz = parameters.Nz
    q = 0.5 * parameters.rho_cruise * parameters.cruise_speed**2

    fuselage_weight = 0.052 * Sf**1.086 * (Nz * Wdg)**0.177 * Lt**(-0.051) * (Lf / D)**(-0.072) * q**0.241

    return fuselage_weight

def compute_horizontal_tail_weight(parameters: AircraftParameters, Wdg: float):

    Aht = parameters.Aht
    Sht = parameters.Sht
    taper_ratio_ht = parameters.taper_ratio_ht
    sweep_ht = np.radians(parameters.sweep_qc_ht)
    tc_ht = parameters.tc_ht
    Nz = parameters.Nz
    q = 0.5 * parameters.rho_cruise * parameters.cruise_speed**2

    ht_weight = 0.016 * (Nz*Wdg)**0.414 * q**0.168 * Sht**0.896 \
        * ( (100 * tc_ht / np.cos(sweep_ht))**-0.12 ) * (Aht / np.cos(sweep_ht)**2)**0.043 * taper_ratio_ht**-0.02
    
    return ht_weight

def compute_vertical_tail_weight(parameters: AircraftParameters, Wdg: float):

    Avt = parameters.Avt
    Svt = parameters.Svt
    taper_ratio_vt = parameters.taper_ratio_vt
    sweep_vt = np.radians(parameters.sweep_qc_vt)
    tc_vt = parameters.tc_vt
    tail_factor = parameters.vtail_factor
    Nz = parameters.Nz
    q = 0.5 * parameters.rho_cruise * parameters.cruise_speed**2

    vt_weight = 0.073 * (1+0.2*tail_factor) * (Nz*Wdg)**0.376 * q**0.122 * Svt**0.873 \
        * (100*tc_vt/np.cos(sweep_vt))**-0.49 * (Avt/np.cos(sweep_vt)**2)**0.357 \
        * taper_ratio_vt**0.039
    
    return vt_weight

def compute_landing_gear_weight(parameters: AircraftParameters, Wdg: float):

    Nl = parameters.Nl
    Lm = parameters.Lm
    WL = 0.95 * Wdg

    lg_weight = 0.054 * Lm**0.501 * (WL*Nl)**0.684

    return lg_weight

def compute_propulsion_weight(parameters: AircraftParameters, Wdg: float):

    engine_weight = parameters.engine_weight
    num_engines = parameters.num_engines
    num_fuel_tanks = parameters.num_fuel_tanks
    fuel_weight = parameters.fuel_weight
    avgas_density = parameters.avgas_density

    Winst_eng = 2.575 * engine_weight**0.922 * num_engines # lbs

    Vt = fuel_weight / avgas_density
    Vi = Vt

    Wfuel_system = 2.49 * Vt**0.726 * (1 + Vi/Vt)**-0.363 * num_fuel_tanks**0.242 * num_engines**0.157 # lbs

    return Winst_eng + Wfuel_system

def compute_miscellaneous_weight(parameters: AircraftParameters, Wdg: float):

    M = parameters.cruise_mach_number
    Kh = parameters.Kh
    num_passengers = parameters.num_passengers

    # flight controls
    Wflight_controls = 0.0168 * Wdg # Roskam Part 5, equation 7.2

    # hydraulics
    M = 0.3 # cruise mach number
    Kh = 0.05 # low subsonic with hydraulics for brakes and retracts only
    Whydraulics = Kh * Wdg**0.8 * M**0.5 # Raymer, equation 15.55

    # avionics
    Wavionics = 40 + 0.008 * Wdg # Roskam Part 5, 7.23

    # electrical
    Welectrical = 0.0268 * Wdg # Roskam Part 5, 7.13

    # furshings
    Wfurshings = 0.412 * num_passengers**1.145 * Wdg**0.489 # Roskam Part 5, 7.41

    return Wflight_controls + Whydraulics + Wavionics + Welectrical + Wfurshings

Below code defines a function for computing empty weight fraction for a given MTOW value, using the functions defined in previous block:

In [2]:
def compute_empty_weight(parameters: AircraftParameters, MTOW: float):
    """
        Function to compute empty weight fraction
    """

    wing_weight = compute_wing_weight(parameters, MTOW)
    fuselage_weight = compute_fuselage_weight(parameters, MTOW)
    ht_weight = compute_horizontal_tail_weight(parameters, MTOW)
    vt_weight = compute_vertical_tail_weight(parameters, MTOW)
    lg_weight = compute_landing_gear_weight(parameters, MTOW)
    propulsion_weight = compute_propulsion_weight(parameters, MTOW)
    misc_weight = compute_miscellaneous_weight(parameters, MTOW)

    empty_weight = wing_weight + fuselage_weight + ht_weight + vt_weight + lg_weight + propulsion_weight + misc_weight

    parameters.wing_weight = wing_weight
    parameters.fuselage_weight = fuselage_weight
    parameters.ht_weight = ht_weight
    parameters.vt_weight = vt_weight
    parameters.lg_weight = lg_weight
    parameters.propulsion_weight = propulsion_weight
    parameters.misc_weight = misc_weight
    parameters.empty_weight = empty_weight

    return empty_weight

Below code block demonstrates how the empty weight function can be used. Note that you need to compute fuel weight before calling empty weight function.

In [3]:
# Parameters
A = 8
S = 134 # sq ft
max_power = 595 # hp
aircraft = AircraftParameters(A, S, max_power)

# Weight
MTOW = 5000 # lbs

fuel_weight = compute_fuel_weight(aircraft, MTOW) # lbs

print(f"Fuel weight for MTOW of {MTOW:.0f} lbs: {fuel_weight:.0f} lbs")

Fuel weight for MTOW of 5000 lbs: 1123 lbs


This concludes the empty weight fraction section, next section describes how these different modules are used for computing MTOW using the iterative process.